# 1. Importing

In [1]:
import gym 
from stable_baselines3 import A2C
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_atari_env
import os

# 2. Testing Environment

In [9]:
# http://www.atarimania.com/roms/Roms.rar ---> download this file and extract it on the folder you are working in order
# to be able to test and use the "Break Out" environment.

In [10]:
!python -m atari_py.import_roms .\ROMS
# You might have a problem with a missing DLL file (ale_c.dll). If that happens I solved this by issuing this command
# on a conda command line: conda install -c conda-forge atari_py

copying adventure.bin from .\ROMS\Adventure (1980) (Atari, Warren Robinett) (CX2613, CX2613P) (PAL).bin to C:\Users\Gabriel Molina\Anaconda3\lib\site-packages\atari_py\atari_roms\adventure.bin

Traceback (most recent call last):
  File "C:\Users\Gabriel Molina\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Gabriel Molina\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Gabriel Molina\Anaconda3\lib\site-packages\atari_py\import_roms.py", line 93, in <module>
    main()
  File "C:\Users\Gabriel Molina\Anaconda3\lib\site-packages\atari_py\import_roms.py", line 89, in main
    import_roms(args.dirpath)
  File "C:\Users\Gabriel Molina\Anaconda3\lib\site-packages\atari_py\import_roms.py", line 74, in import_roms
    with open(filepath, "rb") as f:
FileNotFoundError: [Errno 2] No such file or directory: ".\\ROMS\\Entombed (Maze Chase, Pharaoh's Tomb, Zombie) (1983) (U.S. Games Corporation - Western Technologies, Jeff Corsiglia, Paul Allen Newell, Steven B. Sidley, Tom Sloper) (VC2007) ~.bin"



copying air_raid.bin from .\ROMS\Air Raid (Men-A-Vision) (PAL) ~.bin to C:\Users\Gabriel Molina\Anaconda3\lib\site-packages\atari_py\atari_roms\air_raid.bin
copying alien.bin from .\ROMS\Alien (1982) (20th Century Fox Video Games, Douglas 'Dallas North' Neubauer) (11006) ~.bin to C:\Users\Gabriel Molina\Anaconda3\lib\site-packages\atari_py\atari_roms\alien.bin
copying amidar.bin from .\ROMS\Amidar (1982) (Parker Brothers, Ed Temple) (PB5310) ~.bin to C:\Users\Gabriel Molina\Anaconda3\lib\site-packages\atari_py\atari_roms\amidar.bin
copying assault.bin from .\ROMS\Assault (AKA Sky Alien) (1983) (Bomb - Onbase) (CA281).bin to C:\Users\Gabriel Molina\Anaconda3\lib\site-packages\atari_py\atari_roms\assault.bin
copying asterix.bin from .\ROMS\Asterix (AKA Taz) (07-27-1983) (Atari, Jerome Domurat, Steve Woita) (CX2696) (Prototype).bin to C:\Users\Gabriel Molina\Anaconda3\lib\site-packages\atari_py\atari_roms\asterix.bin
copying asteroids.bin from .\ROMS\Asteroids (1981) (Atari, Brad Stewart

In [2]:
environment_name = "Breakout-v0"

In [3]:
env = gym.make(environment_name)

In [4]:
env.reset() # gives a reset of the environment

array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8)

In [5]:
env.action_space # it is a discrete space with 4 possible actions

Discrete(4)

In [6]:
env.observation_space # probably an image

Box([[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]], [[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
 

In [7]:
episodes = 5
for episode in range (1, episodes+1):
    obs = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action = env.action_space.sample()
        obs, reward, done, info = env.step(action)
        score+=reward
    print('Episode: {}, Score: {}'.format(episode, score))
env.close()
    

C:\Users\Gabriel Molina\Anaconda3\lib\site-packages\pyglet\image\codecs\wic.py:289: UserWarning: [WinError -2147417850] Não é possível alterar o modo de thread depois de o mesmo estar definido
  warnings.warn(str(err))


Episode: 1, Score: 0.0
Episode: 2, Score: 2.0
Episode: 3, Score: 0.0
Episode: 4, Score: 1.0
Episode: 5, Score: 3.0


# 3. Vectorizing and Training Model
 ### We vectorize the environment in order to train in parallel thus improving the speed of agent training.

In [8]:
env = make_atari_env('Breakout-v0', n_envs=9, seed=0)
env = VecFrameStack(env, n_stack=9)

In [9]:
env.render() # The image shows we have 9 environments in parallel.

In [10]:
env.close() # Closing the environment window image.

In [11]:
log_path = os.path.join('Training', 'Logs')
model = A2C('CnnPolicy', env, verbose=1, tensorboard_log=log_path)

Using cpu device
Wrapping the env in a VecTransposeImage.


In [13]:
model.learn(total_timesteps=1000000) # This process took my computer 4 hours.

Logging to Training\Logs\A2C_1
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 290      |
|    ep_rew_mean        | 1.68     |
| time/                 |          |
|    fps                | 66       |
|    iterations         | 100      |
|    time_elapsed       | 67       |
|    total_timesteps    | 4500     |
| train/                |          |
|    entropy_loss       | -1.38    |
|    explained_variance | 0.274    |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | -0.132   |
|    value_loss         | 0.105    |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 284      |
|    ep_rew_mean        | 1.62     |
| time/                 |          |
|    fps                | 68       |
|    iterations         | 200      |
|    time_elapsed       | 131      |
|    total_timesteps    | 9000     |
| train

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 403      |
|    ep_rew_mean        | 4.08     |
| time/                 |          |
|    fps                | 73       |
|    iterations         | 1400     |
|    time_elapsed       | 861      |
|    total_timesteps    | 63000    |
| train/                |          |
|    entropy_loss       | -0.968   |
|    explained_variance | 0.756    |
|    learning_rate      | 0.0007   |
|    n_updates          | 1399     |
|    policy_loss        | -0.0504  |
|    value_loss         | 0.114    |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 421      |
|    ep_rew_mean        | 4.44     |
| time/                 |          |
|    fps                | 73       |
|    iterations         | 1500     |
|    time_elapsed       | 921      |
|    total_timesteps    | 67500    |
| train/                |          |
|

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 495      |
|    ep_rew_mean        | 6.02     |
| time/                 |          |
|    fps                | 74       |
|    iterations         | 2800     |
|    time_elapsed       | 1683     |
|    total_timesteps    | 126000   |
| train/                |          |
|    entropy_loss       | -0.736   |
|    explained_variance | 0.691    |
|    learning_rate      | 0.0007   |
|    n_updates          | 2799     |
|    policy_loss        | 0.14     |
|    value_loss         | 0.114    |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 469      |
|    ep_rew_mean        | 5.5      |
| time/                 |          |
|    fps                | 74       |
|    iterations         | 2900     |
|    time_elapsed       | 1741     |
|    total_timesteps    | 130500   |
| train/                |          |
|

-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 530       |
|    ep_rew_mean        | 7.02      |
| time/                 |           |
|    fps                | 75        |
|    iterations         | 4200      |
|    time_elapsed       | 2493      |
|    total_timesteps    | 189000    |
| train/                |           |
|    entropy_loss       | -0.229    |
|    explained_variance | 0.924     |
|    learning_rate      | 0.0007    |
|    n_updates          | 4199      |
|    policy_loss        | -0.000223 |
|    value_loss         | 0.0515    |
-------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 518      |
|    ep_rew_mean        | 6.82     |
| time/                 |          |
|    fps                | 75       |
|    iterations         | 4300     |
|    time_elapsed       | 2552     |
|    total_timesteps    | 193500   |
| train/             

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 560      |
|    ep_rew_mean        | 7.38     |
| time/                 |          |
|    fps                | 75       |
|    iterations         | 5500     |
|    time_elapsed       | 3287     |
|    total_timesteps    | 247500   |
| train/                |          |
|    entropy_loss       | -0.0753  |
|    explained_variance | 0.705    |
|    learning_rate      | 0.0007   |
|    n_updates          | 5499     |
|    policy_loss        | 0.0161   |
|    value_loss         | 0.185    |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 573      |
|    ep_rew_mean        | 7.61     |
| time/                 |          |
|    fps                | 75       |
|    iterations         | 5600     |
|    time_elapsed       | 3349     |
|    total_timesteps    | 252000   |
| train/                |          |
|

...

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 692      |
|    ep_rew_mean        | 10.8     |
| time/                 |          |
|    fps                | 76       |
|    iterations         | 16400    |
|    time_elapsed       | 9708     |
|    total_timesteps    | 738000   |
| train/                |          |
|    entropy_loss       | -0.15    |
|    explained_variance | 0.614    |
|    learning_rate      | 0.0007   |
|    n_updates          | 16399    |
|    policy_loss        | 0.0166   |
|    value_loss         | 0.198    |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 679      |
|    ep_rew_mean        | 10.7     |
| time/                 |          |
|    fps                | 76       |
|    iterations         | 16500    |
|    time_elapsed       | 9767     |
|    total_timesteps    | 742500   |
| train/                |          |
|

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 726      |
|    ep_rew_mean        | 11.7     |
| time/                 |          |
|    fps                | 76       |
|    iterations         | 17800    |
|    time_elapsed       | 10527    |
|    total_timesteps    | 801000   |
| train/                |          |
|    entropy_loss       | -0.18    |
|    explained_variance | 0.818    |
|    learning_rate      | 0.0007   |
|    n_updates          | 17799    |
|    policy_loss        | -0.00702 |
|    value_loss         | 0.187    |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 729      |
|    ep_rew_mean        | 11.6     |
| time/                 |          |
|    fps                | 76       |
|    iterations         | 17900    |
|    time_elapsed       | 10586    |
|    total_timesteps    | 805500   |
| train/                |          |
|

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 730      |
|    ep_rew_mean        | 12       |
| time/                 |          |
|    fps                | 68       |
|    iterations         | 19200    |
|    time_elapsed       | 12530    |
|    total_timesteps    | 864000   |
| train/                |          |
|    entropy_loss       | -0.146   |
|    explained_variance | 0.864    |
|    learning_rate      | 0.0007   |
|    n_updates          | 19199    |
|    policy_loss        | 0.0284   |
|    value_loss         | 0.0581   |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 719      |
|    ep_rew_mean        | 11.6     |
| time/                 |          |
|    fps                | 68       |
|    iterations         | 19300    |
|    time_elapsed       | 12588    |
|    total_timesteps    | 868500   |
| train/                |          |
|

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 769      |
|    ep_rew_mean        | 13.6     |
| time/                 |          |
|    fps                | 69       |
|    iterations         | 20500    |
|    time_elapsed       | 13331    |
|    total_timesteps    | 922500   |
| train/                |          |
|    entropy_loss       | -0.162   |
|    explained_variance | 0.638    |
|    learning_rate      | 0.0007   |
|    n_updates          | 20499    |
|    policy_loss        | -0.0919  |
|    value_loss         | 0.329    |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 769      |
|    ep_rew_mean        | 13.6     |
| time/                 |          |
|    fps                | 69       |
|    iterations         | 20600    |
|    time_elapsed       | 13394    |
|    total_timesteps    | 927000   |
| train/                |          |
|

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 789      |
|    ep_rew_mean        | 13.3     |
| time/                 |          |
|    fps                | 69       |
|    iterations         | 21800    |
|    time_elapsed       | 14147    |
|    total_timesteps    | 981000   |
| train/                |          |
|    entropy_loss       | -0.281   |
|    explained_variance | 0.76     |
|    learning_rate      | 0.0007   |
|    n_updates          | 21799    |
|    policy_loss        | -0.0557  |
|    value_loss         | 0.206    |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 783      |
|    ep_rew_mean        | 13.1     |
| time/                 |          |
|    fps                | 69       |
|    iterations         | 21900    |
|    time_elapsed       | 14211    |
|    total_timesteps    | 985500   |
| train/                |          |
|

# 4. Save and Reload Model

In [26]:
a2c_path = os.path.join('Training', 'Saved Models', 'A2C_model')

In [27]:
model.save(a2c_path)

NameError: name 'model' is not defined

In [28]:
del model

NameError: name 'model' is not defined

In [37]:
env = make_atari_env('Breakout-v0', n_envs=1, seed=0) # n_envs=1 for testing we can use just one environment.
env = VecFrameStack(env, n_stack=9) # n_stack must continue 9 because it is how the model was trained.

In [38]:
model = A2C.load(a2c_path, env)

Wrapping the env in a VecTransposeImage.


# 5 Evaluate and testing

In [39]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

(15.9, 5.048762224545735)

In [40]:
env.close()